In [ ]:
# Makes sure nested parallelism is avoided across any underlying implementation
import os
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
#multiprocessing.set_start_method('spawn', force=True)
from multiprocessing import Pool

def create_localiser(L,rho,kappa,disorder):
    # Function to create the localiser matrix of a 3D Anderson model
    # Note the open boundary conditions
    # This also includes the creation of the position operator X
    t = 1

    # on diagonal disorder
    diag = (np.random.rand(L**3)-0.5) * disorder
    # off diagonal hopping term
    X1_diag = np.tile(np.append(t * np.ones(L-1) ,0),               L**2)
    X2_diag = np.tile(np.append(np.ones(L*(L-1)) ,np.zeros(L)),     L)
    X3_diag = np.append(np.ones(L*L*(L-1)) ,np.zeros(L*L))
    H = sp.diags([X3_diag,X2_diag,X1_diag,X1_diag,X2_diag,X3_diag],[-L*L,-L,-1,1,L,L*L],shape=(L**3,L**3),format='lil')

    row_vector = np.linspace(-rho,rho,L)
    X1 = sp.diags(np.tile(row_vector,L**2),0,shape=(L**3,L**3),format='csr')
    X2 = sp.diags(np.tile(np.repeat(row_vector,L),L),0,shape=(L**3,L**3),format='csr')
    X3 = sp.diags(np.repeat(row_vector,L**2),0,shape=(L**3,L**3),format='csr')

    #H = sp.diags([off_diag,diag,off_diag],[-1,0,1],shape=(L,L),format='lil')

    #H[L-1,0] = 1
    #H[0,L-1] = 1

    row_vector = np.linspace(-rho,rho,L)
    X = sp.diags(row_vector,0,shape=(L,L),format='csr')

    localiser = sp.bmat([[-H,kappa * X],[kappa * X,H]],format='csr')

    return localiser


def find_eigenvalues(localiser, num_eigenvalues=800):
    # Function to find the positive eigenvalues of the localiser matrix
    # numpy eigsh is best for the sparse case.
    eigvals, eigvecs = eigsh(localiser, k=num_eigenvalues,sigma = 0, which='LM')
    #eigvals, eigvecs = eigsh(localiser, k=num_eigenvalues, which='SM')
    positive_eigvals = eigvals[eigvals > 0]
    return positive_eigvals



# Calculate the adjacent gap ratio r = min(s_i,s_(i+1))/max(s_i,s_(i+1))
def calculate_r(eigvals):
    # Once eigenvalues are found, calculate the r value
    eigvals_s = np.diff(eigvals)
    min_eigvals_s = np.array([min(eigvals_s[i],eigvals_s[i+1]) for i in range(len(eigvals_s)-1)])
    max_eigvals_s = np.array([max(eigvals_s[i],eigvals_s[i+1]) for i in range(len(eigvals_s)-1)])
    r = min_eigvals_s / max_eigvals_s
    return r.mean()


def calculate_z(eigvals):    
    eigvals = sorted(eigvals)
    z = np.zeros(len(eigvals)-4)
    for i in range(2,len(eigvals)-2):
        if abs(eigvals[i+1] - eigvals[i]) < abs(eigvals[i]-eigvals[i-1]):
            nn = abs(eigvals[i+1] - eigvals[i])
            nnn = min(min(abs(eigvals[i]-eigvals[i-1]),abs(eigvals[i+2]-eigvals[i])),abs(eigvals[i-2]-eigvals[i]))
        else:
            nn = abs(eigvals[i]-eigvals[i-1])
            nnn = min(min(abs(eigvals[i+1]-eigvals[i]),abs(eigvals[i-2]-eigvals[i])),abs(eigvals[i+2]-eigvals[i]))
        z[i-2] = nn/nnn
    return z.mean()

In [ ]:
import numpy as np
t = 1
L = 3
X1_diag = np.tile(np.append(t * np.ones(L-1) ,0),               L**2)
X2_diag = np.tile(np.append(np.ones(L*(L-1)) ,np.zeros(L)),     L)
X3_diag = np.append(np.ones(L*L*(L-1)) ,np.zeros(L*L))
print(X3_diag)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]


In [ ]:

import scipy.sparse as sp
import numpy as np
rho = 1
L = 3



print(X3_dense.diagonal())

[-1. -1. -1. -1. -1. -1. -1. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  1.  1.  1.  1.  1.  1.  1.  1.]
